<a href="https://colab.research.google.com/github/JacopoMangiavacchi/animegan2-coreml/blob/master/convert_coreml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install coremltools

In [2]:
import os
import torch
import coremltools

In [3]:
!git clone https://github.com/jacopomangiavacchi/animegan2-coreml

Cloning into 'animegan2-coreml'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 55 (delta 16), reused 34 (delta 5), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [4]:
os.chdir(f'./animegan2-coreml')

In [5]:
from model import Generator

device = 'cpu'
torch.set_grad_enabled(False)

model = Generator().eval().to(device)

In [6]:
ckpt = torch.load(f"weights/celeba_distill.pt", map_location=device)
# ckpt = torch.load(f"weights/face_paint_512_v2.pt", map_location=device)
# ckpt = torch.load(f"weights/paprika.pt", map_location=device)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [7]:
image_size = 512

image = torch.randn(1, 3, image_size, image_size)
output = model(image.to(device))

In [8]:
# scripted_model = torch.jit.script(model)
traced = torch.jit.trace(model, image)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2359: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  _verify_batch_size([input.size(0) * input.size(1) // num_groups, num_groups] + list(input.size()[2:]))


In [9]:
image_input = coremltools.ImageType(name="input_1",
                           shape=(1, 3, image_size, image_size))

In [10]:
mlmodel = coremltools.converters.convert(
  traced,
  inputs=[image_input],
)

Running MIL Common passes:   0%|          | 0/34 [00:00<?, ? passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '457', of the source model, has been renamed to 'var_457' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 802/802 [00:00<00:00, 2297.52 ops/s]


In [11]:
mlmodel.save(f'animegan2-celeba-distil-{image_size}.mlmodel')

In [12]:
spec = coremltools.utils.load_spec(f"animegan2-celeba-distil-{image_size}.mlmodel")

In [13]:
print(spec.description.input)

[name: "input_1"
type {
  imageType {
    width: 512
    height: 512
    colorSpace: RGB
  }
}
]


In [14]:
print(spec.description.output)

[name: "var_457"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]


In [15]:
output = spec.description.output[0]

output.type.imageType.colorSpace = coremltools.proto.FeatureTypes_pb2.ImageFeatureType.RGB
output.type.imageType.height = image_size
output.type.imageType.width = image_size

coremltools.utils.save_spec(spec, f"animegan2-celeba-distil-{image_size}.mlmodel")

In [16]:
spec = coremltools.utils.load_spec(f"animegan2-celeba-distil-{image_size}.mlmodel")

In [17]:
print(spec.description.output)

[name: "var_457"
type {
  imageType {
    width: 512
    height: 512
    colorSpace: RGB
  }
}
]


In [18]:
os.chdir(f'..')